<a href="https://colab.research.google.com/github/mauromelo-ti-ifce/PPGCC/blob/main/n1_am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import csv
import random
import math
import operator

def loadDataset(iris, split, trainingSet=[] , testSet=[]):
    with open(iris, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

 def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0   

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

# Função para carregar os dados da coluna vertebral
def load_vertebral_column_uci():
    # Seus dados aqui
    data = {
        'pelvic_incidence': [63.03, 39.06, 68.83, 69.3, 49.71],
        'pelvic_tilt': [22.55, 10.06, 22.22, 24.65, 9.65],
        'lumbar_lordosis_angle': [39.61, 25.02, 50.09, 44.31, 28.32],
        'sacral_slope': [40.48, 29, 46.61, 44.64, 40.06],
        'pelvic_radius': [98.67, 114.41, 105.99, 101.87, 108.17],
        'degree_spondylolisthesis': [-0.25, 4.56, -3.53, 11.21, 7.92],
        'class': ['DH', 'DH', 'DH', 'DH', 'DH']
    }
    df = pd.DataFrame(data)
    return df

# Carregar os dados
vertebral_data = load_vertebral_column_uci()

# Dividir os dados em features (X) e target (y)
X = vertebral_data.drop(columns=['class'])
y = vertebral_data['class']

# Inicializar o classificador Naive Bayes Gaussiano
gnb = GaussianNB()

# Número de execuções
num_execucoes = 20

acuracias = []

for _ in range(num_execucoes):
    # Dividir os dados em conjunto de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=_)

    # Treinar o classificador
    gnb.fit(X_train, y_train)

    # Fazer previsões
    y_pred = gnb.predict(X_test)

    # Calcular a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    acuracias.append(accuracy)

    # Calcular e imprimir a matriz de confusão para a última execução
    if _ == num_execucoes - 1:
        conf_matrix = confusion_matrix(y_test, y_pred)
        print("\nMatriz de Confusão (última execução):")
        print(conf_matrix)

# Calcular a acurácia média e o desvio padrão da acurácia
acuracias = np.array(acuracias)
acuracia_media = np.mean(acuracias)
desvio_padrao_acuracia = np.std(acuracias)

print("\nAcurácia Média ({} execuções):".format(num_execucoes), acuracia_media)
print("Desvio Padrão da Acurácia ({} execuções):".format(num_execucoes), desvio_padrao_acuracia)



Matriz de Confusão (última execução):
[[1]]

Acurácia Média (20 execuções): 1.0
Desvio Padrão da Acurácia (20 execuções): 0.0


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from knn_dmc import KNN, DMC
from slave import GaussianNB
import requests
import zipfile
import os
import pandas as pd

# Função para carregar o conjunto de dados vertebral_column
def load_vertebral_column_uci():
    # URL do arquivo ZIP
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip"
    # Caminho local para salvar o arquivo ZIP
    zip_path = "vertebral_column_data.zip"
    # Caminho local para o arquivo de dados extraído
    data_path = "column_3C.dat"

    # Baixar o arquivo ZIP se ainda não foi baixado
    if not os.path.exists(zip_path):
        r = requests.get(url)
        with open(zip_path, "wb") as f:
            f.write(r.content)

    # Extrair o arquivo de dados do ZIP se ainda não foi extraído
    if not os.path.exists(data_path):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall()

    # Ler o arquivo de dados
    column_names = ['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'class']
    vertebral_data = pd.read_csv(data_path, header=None, sep=' ', names=column_names)

    # Convertendo a coluna 'class' para valores numéricos
    class_mapping = {'DH': 0, 'SL': 1, 'NO': 2}
    y = vertebral_data['class'].map(class_mapping).values
    X = vertebral_data.iloc[:, :-1].values

    return X, y

# Função para calcular a acurácia
def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Função para calcular a matriz de confusão
def confusion_matrix(y_true, y_pred):
    classes = np.unique(y_true)
    num_classes = len(classes)
    matrix = np.zeros((num_classes, num_classes), dtype=int)
    for i in range(num_classes):
        for j in range(num_classes):
            matrix[i, j] = np.sum((y_true == classes[i]) & (y_pred == classes[j]))
    return matrix

# Função para separar os dados em conjunto de treinamento e teste
def train_test_split(X, y, test_size=0.3, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    test_samples = int(len(X) * test_size)
    X_train = X[indices[:-test_samples]]
    X_test = X[indices[-test_samples:]]
    y_train = y[indices[:-test_samples]]
    y_test = y[indices[-test_samples:]]
    return X_train, X_test, y_train, y_test

# Função para realizar 20 realizações de um classificador para uma base de dados
def perform_realizations(classifier, X, y):
    accuracies = []
    confusion_matrices = []

    for _ in range(20):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=_)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        accuracies.append(acc)
        confusion_matrices.append(conf_matrix)

    accuracies = np.array(accuracies)
    mean_accuracy = np.mean(accuracies)
    std_dev_accuracy = np.std(accuracies)
    best_realization_idx = np.argmin(np.abs(accuracies - mean_accuracy))
    best_confusion_matrix = confusion_matrices[best_realization_idx]

    return mean_accuracy, std_dev_accuracy, best_realization_idx, best_confusion_matrix


# Função para visualizar as superfícies de decisão
def plot_decision_surface(clf, X_train, y_train, title):
    # Escolha dos dois primeiros atributos
    X_train = X_train[:, :2]

    # Treinar o classificador com os dois primeiros atributos
    clf.fit(X_train, y_train)

    # Criação da grade de pontos
    h = .02  # tamanho do passo na grade
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Predição para cada ponto na grade
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Colocando o resultado na mesma forma que a grade
    Z = Z.reshape(xx.shape)

    # Plotagem das fronteiras de decisão
    plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.show()

# Execução da análise para o conjunto de dados vertebral_column
print("\nDataset: Vertebral Column")

X, y = load_vertebral_column_uci()

knn_classifier = KNN()
dmc_classifier = DMC()
naive_bayes_classifier = GaussianNB()  # Este é o Naive Bayes implementado anteriormente

classifiers = {"KNN": knn_classifier, "DMC": dmc_classifier, "Naive Bayes": naive_bayes_classifier}

# Listas para armazenar os resultados de acurácia e desvio padrão
accuracies = []
std_devs = []

for clf_name, clf in classifiers.items():
    print(f"\nClassificador: {clf_name}")

    # Visualização da superfície de decisão
    plot_decision_surface(clf, X, y, f'Vertebral Column - Decision Surface for {clf_name}')

    # Realização da análise
    mean_acc, std_dev_acc, best_realization_idx, best_conf_matrix = perform_realizations(clf, X, y)

    # Impressão dos resultados
    print(f"Acurácia média: {mean_acc}")
    print(f"Desvio padrão da acurácia: {std_dev_acc}")
    print(f"Melhor realização (índice): {best_realization_idx}")
    print(f"Matriz de confusão da melhor realização:\n{best_conf_matrix}")

ModuleNotFoundError: No module named 'ucimlrepo'

In [4]:
!git clone https://github.com/mauromelo-ti-ifce/class_ifce.git


Cloning into 'class_ifce'...
fatal: could not read Username for 'https://github.com': No such device or address
